In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("data/南海_2018-01-01_2020-02-10.csv")

In [7]:
original_text_result = df[df['original_text'].str.contains('海军|南海|航行')] #找到'original_text'列中包含'南海'的所有行 
original_text_result

,person_name,org_name,pos,verb,viewpoint,person_id,org_id,news_id,sentiment,time,publish_time,original_text,per_count,org_count,pos_count,entity_importance,org_pos
194,NaN,美国,海军,说,1月11日至16日期间，驻扎在日本的美军导弹驱逐舰“麦坎贝尔”号和正在亚洲巡回访问的英国皇家...,-1,31912,8887143353463512863,0.870405,2019-01-17T11:33:51.412Z,2019-01-17T12:58:00.000000Z,原标题：挑衅升级：美英军舰首次在南海联合演习！ 挑衅升级：美英军舰首次在南海联合演习！ 来自...,-1,55840,-1.00,35.7376,美国海军
195,NaN,美国,海军发言人,称,“具体到南海地区，两军近年来没有关于联合行动的任何记录。”他补充说，至少自2010年以来两国...,-1,31912,8887143353463512863,0.991305,2019-01-17T11:33:51.412Z,2019-01-17T12:58:00.000000Z,原标题：挑衅升级：美英军舰首次在南海联合演习！ 挑衅升级：美英军舰首次在南海联合演习！ 来自...,-1,55840,-1.00,35.7376,美国海军发言人
300,罗纳德·博索尔,NaN,NaN,透露,无人自主作战艇将使美军水面舰艇告别更大、更贵的建造方式，转而发展中小型无人自主作战舰艇。,284250,-1,6015225729952292086,0.999960,2019-01-17T08:40:42.807Z,2019-01-17T12:50:04.000000Z,载人舰艇凑不够数，无人化舰艇成为美海军的救命稻草。《海军时报》15日称，由美国国防部高级研究...,-1,-1,-1.00,3.0000,NaN
503,NaN,美国,“商业内幕网,称,尽管美军研发电磁炮时间要早于中国，但尚未将其安装到战舰上进行集成化测试，中国海军“海洋山”号...,-1,31912,7794960855200685401,0.999999,2019-01-17T08:52:46.565Z,2019-01-17T00:00:00.000000Z,另据美国“商业内幕网”1月1日报道称，尽管美军研发电磁炮时间要早于中国，但尚未将其安装到战舰...,-1,55840,-1.00,35.7376,美国“商业内幕网
504,NaN,日本,NaN,称,舰载电磁轨道炮可利用电磁力产生动能，将弹丸加速至7马赫以上的高超音速，射程达到200公里以上...,-1,31891,7794960855200685401,0.040284,2019-01-17T08:52:46.565Z,2019-01-17T00:00:00.000000Z,据日本《读卖新闻》1月13日报道称，舰载电磁轨道炮可利用电磁力产生动能，将弹丸加速至7马赫以...,-1,7296,-1.00,466.9440,日本
505,NaN,美国,“任务与意志”网站,称,与普通072II型坦克登陆舰不同，“海洋山”号登陆舰的舰艏拆除了原先的双37毫米防空炮，取而...,-1,31912,7794960855200685401,0.114683,2019-01-17T08:52:46.565Z,2019-01-17T00:00:00.000000Z,据美国“任务与意志”网站报道称，与普通072II型坦克登陆舰不同，“海洋山”号登陆舰的舰艏拆...,-1,55840,-1.00,35.7376,美国“任务与意志”网站
716,NaN,日本,NaN,称,中国军队正在加速研发新式武器，其中就包括舰载电磁轨道炮（下文简称“电磁炮”），中国海军工程大...,-1,31891,8028428704191370394,0.990976,2019-01-17T11:09:48.575Z,2019-01-17T13:51:00.000000Z,日本称中国军队正在加速研发新式武器，其中就包括舰载电磁轨道炮（下文简称“电磁炮”），中国海军...,-1,7296,-1.00,466.9440,日本
785,彭斯,美国,副总统,说,中国过去几年来经常无视国际法和国际规范，美国不会再对此视而不见。,128092,31912,3069401096039330263,0.293096,2019-01-17T09:38:05.693Z,2019-01-16T19:16:45.000000Z,美国副总统彭斯在国务院的一次会议上多次提及中国，指责中国无视国际规范。 综合媒体1月16日报...,46,55840,24.00,46.0000,美国副总统
867,NaN,美国海军研究学会,网站,称,近年来，美国海军的实有军舰总数在270至290艘之间上下波动。,-1,129199,4863669036824905721,0.963381,2019-01-17T09:41:14.787Z,2019-01-17T09:57:38.000000Z,美国海军研究学会网站刊文称，近年来，美国海军的实有军舰总数在270至290艘之间上下波动。造...,-1,9,-1.00,0.0090,美国海军研究学会网站
882,NaN,美国,海军,称,美国海军导弹驱逐舰麦坎贝尔号（USSMcCampbell）与英国皇家海军护卫舰阿吉尔号（HM...,-1,31912,2608855714515825122,0.004062,2019-01-17T09:11:04.006Z,2019-01-16T06:42:06.000000Z,据路透社2019年1月16日报道，美国海军在新闻稿中称，美国海军导弹驱逐舰麦坎贝尔号（USS...,-1,55840,-1.00,35.7376,美国海军


In [9]:
viewpoint_result = df[df['viewpoint'].str.contains('海军|南海|航行')] #找到'original_text'列中包含'南海'的所有行 
viewpoint_result